# Baseline RAG - landsforsøg 2022 pdf

## Setup

In [96]:
#%pip install llama-index-readers-smart-pdf-loader
#%pip install llama-parse
#%pip install llmsherp
#%pip install --upgrade chromadb==0.4.14

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 448.1/448.1 kB 5.6 MB/s eta 0:00:000:00:01m eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.0/11.0 MB 11.6 MB/s eta 0:00:00m eta 0:00:010:00:01
  Attempting uninstall: chromadb
    Found existing installation: chromadb 0.5.0
    Uninstalling chromadb-0.5.0:
      Successfully uninstalled chromadb-0.5.0
Note: you may need to restart the kernel to use updated packages.


In [3]:
import os
import chromadb
import chromadb.utils.embedding_functions as embedding_functions
from chromadb import Settings
from IPython.display import Markdown, display

from llama_index.core import SimpleDirectoryReader, VectorStoreIndex
from llama_index.core import PromptTemplate, SimpleDirectoryReader

from llama_index.core.node_parser import SentenceSplitter
from llama_index.core.ingestion import IngestionPipeline
from llama_index.vector_stores.chroma import ChromaVectorStore
from openai import OpenAI, AzureOpenAI

from dotenv import load_dotenv

from llmsherpa.readers import LayoutPDFReader
from llama_index.readers.smart_pdf_loader import SmartPDFLoader

#from llama_index.llms.azure_openai import AzureOpenAI
#rom llama_index.embeddings.azure_openai import AzureOpenAIEmbedding


#import importlib
#import util

#importlib.reload(util.helpers)
#from util.helpers import create_and_save_md_files, get_malazan_pages, get_office_pages, get_friends_pages, get_theoffice_pages

In [4]:
load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
AZURE_OPENAI_ENDPOINT = os.getenv("AZURE_OPENAI_ENDPOINT")

In [8]:
# ChromaDB Vector Store
chroma_client = chromadb.PersistentClient(
    path="./chromadb", settings=Settings(allow_reset=True))

openai_client = AzureOpenAI(
    api_key=OPENAI_API_KEY,  
    api_version="2024-05-01-preview", # https://learn.microsoft.com/en-us/azure/ai-services/openai/reference?WT.mc_id=AZ-MVP-5004796
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT")
)

openai_ef = embedding_functions.OpenAIEmbeddingFunction(
    api_key=OPENAI_API_KEY,
    model_name="text-embedding-ada-002",
    api_type="azure",
    api_version="2024-05-01-preview"
)

## Load document(s)

### Attempt 3: SimpleDirectoryReader

In [10]:
reader = SimpleDirectoryReader("./documents/")
#/Husdyrbrugloven.pdf","landsforsøg/documents/FT-73_Klovvaskere_web.pdf
documents = reader.load_data()

In [ ]:
#authentication error
#index = VectorStoreIndex.from_documents(documents)

In [11]:
document_data = []
for idx, doc in enumerate(documents):
    embedding = openai_client.embeddings.create(
        input=doc.text, model="text-embedding-ada-002"
    )
    document_data.append({
        "id": f"{doc.id_}-{idx}",
        "text": doc.text,
        "metadata":doc.metadata,
        "embedding": embedding.data[0].embedding
    })


In [12]:
documents = [doc["text"] for doc in document_data]
embeddings = [doc["embedding"] for doc in document_data]
metadatas = [doc["metadata"] for doc in document_data]
ids = [doc["id"] for doc in document_data]

In [13]:
chroma_client.reset()
collection = chroma_client.get_or_create_collection(
    name="tesla", metadata={"hnsw:space": "cosine"}, embedding_function=openai_ef)

In [15]:
collection.add(
    embeddings=embeddings,
    documents=documents,
    metadatas=metadatas,
    ids=ids)

### Retrieval

In [17]:
query = "hvordan ændrer jeg på temperaturen i min Tesla"

In [ ]:
result = collection.query(query_texts=[query], n_results=5)
context = result["documents"][0]
#display(Markdown(f"------------\n\n{"\n\n------------\n\n".join(context)}"))

formatted_text = "\n\n------------\n\n".join(context)

# Display the formatted markdown
display(Markdown(f"{formatted_text}"))

## Generation

In [24]:

prompt = PromptTemplate("""You are a helpful assistant that answers questions about my Tesla Model y using provided context from the owner's manual. You must provide your answer in the Danish language. You should provide the number of the page containing the information given in the answer.

Question: {query}

Context: 

-----------------------------------
{context}

-----------------------------------

""")
message = prompt.format(query=query, context="\n\n".join(context))
display(Markdown(f"{message}"))

You are a helpful assistant that answers questions about my Tesla Model y using provided context from the owner's manual. You must provide your answer in the Danish language. You should provide the number of the page containing the information given in the answer.

Question: hvordan ændrer jeg på temperaturen i min Tesla

Context: 

-----------------------------------
Klimaanlæg
Du kan kontrollere temperaturen i kabinen og opvarme eller 
afkøle kabinen, før du kører (også selv om den står i en 
garage), styre sædevarmen og afrime forruden:
• Aktiver eller deaktiver Afrim bil , som hjælper med at 
smelte sne, is og frost på forrude, vinduer og spejle, ved 
at stryge op fra bunden af skærmen.
• Aktiver eller deaktiver Kæledyrstilstand  eller Camp-
tilstand .
• Aktiver Overophedningsbeskyttelse for kabinen  som 
forhindrer, at kabinen bliver for varm i varme omgivende 
temperaturer. Du kan vælge, om du vil have 
airconditionanlægget eller blot ventilatoren til at køre, når 
temperaturen i kabinen overstiger 40° C , eller den valgte 
temperatur (hvis den er tilgængelig). Se Betjening af 
klimaanlægget  på side 154  for yderligere oplysninger.
• Åbne eller lukke ruderne.
• Klargør kabinen til den ønskede temperatur, og tænd eller 
sluk for rat og sædevarmen (ekstraudstyr).
Brug af mobilappen til at forvarme Model Y  opvarmer også 
batteriet efter behov. Mobilappen giver dig besked, når din 
bil har nået den ønskede klargøringstemperatur.
BEMÆRK:  Afhængigt af køretøjsspecifikationerne og 
produktionsdatoen gælder det for nogle køretøjer, at brug af 
mobilappen til at afrime Model Y  også optør is på 
ladeportens lås. Dette er nyttigt i ekstremt koldt vejr eller 
frostvejr, hvor ladeportens lås kan fryse fast, så du ikke kan 
tage ladekablet ud eller sætte det i.
Placering
Finde Model Y  med anvisninger eller spore dens rute på et 
kort.
Summon
Du kan parkere eller hente Model Y  vha. Summon (se 
Summon  på side 127 ) eller Smart Summon (se Smart 
Summon  på side 129 ).
Planlæg
Aktiver planlagt opladning eller afrejse, og klargør køretøjet. 
Se Planlagt opladning og Planlagt afrejse  på side 184  for 
yderligere oplysninger. Planlagt opladning eller afrejse kan 
også gemmes baseret på en foretrukken placering.
Sikkerhed
Sikkerhedsfanen lader dig gøre følgende:
• Parre din telefon med bilen (se Telefonnøgle  på side 22 ).• Aktivere eller deaktivere Sentry-tilstand (se Sådan bruger 
du Sentry-tilstand (med et USB-flashdrev)  på side 150 ).
• Aktivere eller deaktivere Valet-tilstand (se Valet-tilstand 
på side 93 ).
• Aktivere eller deaktivere 
Hastighedsbegrænsningsfunktion og modtage beskeder, 
hvis køretøjets kørehastighed er ca. 5 km/t  fra den valgte 
maksimale hastighed (se 
Hastighedsbegrænsningsfunktion  på side 146 ).
Opgraderinger
Se og køb de nyeste opgraderinger til din bil, som f.eks. Full 
Self-Driving.
Service
Se Bestil service  på side 189  for oplysninger om, hvordan 
service bestilles via mobilappen.
Vejside
Se vejhjælpsressourcer og anmode om vejhjælp (hvor det er 
relevant). For yderligere oplysninger om vejhjælp, se Opkald 
til Teslas vejhjælp  på side 222 .
Give tilladelse til en sekundær fører
Tilføj og fjern adgangstilladelse for en ekstra fører fra Tesla-
mobilappen.
BEMÆRK:  Tesla-mobilapp version 4.3.1 eller nyere er 
påkrævet. Ekstra førere kan enten benytte en tidligere 
registreret Tesla-konto eller benytte appen til at oprette en 
ny Tesla-konto.
For at tilføje en ekstra fører skal du i Tesla-mobilappen fra 
køretøjets startskærm gå til Sikkerhed  > Tilføj fører  og følge 
instruktionerne på skærmen.
BEMÆRK:  Den ekstra fører har adgang til alle app-funktioner 
undtagen køb af opgraderinger.
For at fjerne adgangen skal du benytte mobilappen og gå til 
Sikkerhed  > Administrer førere  og følge instruktionerne på 
skærmen.Mobilapp
58 MODEL Y Instruktionsbog

Pop op-vindue for klimaanlægget
Tryk på temperaturpilene nederst på touchskærmen for at få 
vist et pop op-vindue for nem adgang til de mest hyppigt 
anvendte indstillinger af klimaanlægget:
BEMÆRK:  For at få adgang til sædevarmere og afrimere med 
ét tryk kan du tilføje disse betjeningsknapper til Mine apper. 
Se Tilpasning af Mine apper  på side 6 .
1. Tryk for at åbne klimaanlæggets hovedskærm.
2. Aktiver eller deaktiver opvarmning af sæder.
3. Aktiver eller deaktiver for- eller bagrudeafrimere.
4. Skift temperaturen i kabinen ved at trække i skyderen. Du 
kan også aktivere temperaturopdeling, som gør det muligt 
for føreren og forsædepassageren at tilpasse efter deres 
egne præferencer. Forsædepassageren kan trykke på 
temperaturikonet i bunden af touchskærmen eller på 
hovedskærmen for klimaanlægget for at justere. Tryk på 
Opdel  igen for at deaktivere temperaturopdeling.
Hold klimaanlæg tændt, Kæledyr og 
Camp
Med indstillingerne Hold klimaanlæg tændt , Kæledyr  og 
Camp  kan du lade klimaanlægget køre, mens bilen er i 
parkeringsgear, selv efter du har forladt din Model Y  eller 
vælger at blive inde i bilen. Disse indstillinger er nyttige, når 
det er vigtigt at opretholde temperaturen i kabinen i varmt 
eller koldt vejr. Hvis du for eksempel har fødevarer i din 
Model Y  på varme dage, kan du benytte indstillingen Hold 
klimaanlæg tændt for at undgå, at varerne bliver fordærvede.
Tilstanden Kæledyr er designet til at opretholde en behagelig 
temperatur i kabinen for dit kæledyr, mens du aktivt og 
hyppigt overvåger temperaturen med mobilappen (hvilket 
kræver, at både din telefon og bilen har mobildækning). I 
tilstanden Kæledyr viser touchskærmen den aktuelle 
kabinetemperatur for at oplyse forbipasserende om, at 
kæledyret har det godt. Indstillingen er ikke beregnet til 
mennesker og må kun bruges i korte perioder, mens du er i 
nærheden, så du kan nå bilen i situationer, hvor temperaturen 
ikke længere kan opretholdes.
BEMÆRK:  For at undgå utilsigtet aktivering af 
vindueskontakten (f.eks. ved at din hund træder på den), kan 
vinduerne ikke rulles ned, når Kæledyrstilstand er aktiveret.BEMÆRK:  Hvis Dog og Sentry er aktiveret på samme tid, er 
Sentrys standard Deaktiver Sentry-lyde  for at beskytte dit 
kæledyr. Se Sentry-tilstand  på side 150  for yderligere 
oplysninger.
Live-kameravisning er nu tilgængelig, hvis Sentry-tilstand 
og/eller Kæledyrstilstand er aktiveret. Hvis Sentry-tilstand er 
aktiveret, viser kameraerne en live-visning af køretøjets 
omgivelser. Hvis Kæledyrstilstand er aktiveret, viser det 
indvendige kabinekamera køretøjet indefra, så du hele tiden 
kan holde øje med dit kæledyr. Hvis begge tilstande er 
aktiverede, kan du skifte kameravisning ved at trykke på de 
grå cirkler eller det ikon, der svarer til de forskellige 
indvendige kameraer i mobilappen. Se Sentry-tilstand  på side 
150 for yderligere oplysninger.
BEMÆRK:  Aktivering af det indvendige kabinekamera til 
Kæledyrstilstand eller Sentry-tilstand kræver mobilappen 
version 4.15.0 eller senere. Denne funktion understøttes ikke 
i køretøjer med Autopilot-computer 2.0 eller 2.5. Tryk på 
Betjeningsknapper  > Software  > Autopilot-computer  for at 
finde ud af, hvilken computer dit køretøj er udstyret med.
BEMÆRK:  Vis live-kamera  er begrænset til ca. 15 minutters 
samlet brug om dagen.
Camp gør, at du kan oplade elektroniske enheder via USB-
portene og lavspænding -udtaget samtidig med, at 
temperaturen i kabinen opretholdes. Touchskærmen bliver 
ved med at være tændt, så du kan afspille musik, browse på 
internettet, spille spil i Arcade eller se programmer via Tesla 
Theater. Du kan også styre medie- og klimaindstillingerne fra 
en parret telefon. Camp er perfekt, når du skal blive inde i 
bilen, for eksempel ved campering eller når du skal blive 
sammen med et barn. Når funktionen er aktiv, er Sentry-
tilstand og bilens alarmsystem deaktiveret. Funktionen 
Låsning af døre, når bilen forlades er inaktiv.
Gør følgende for at betjene Hold klimaanlæg tændt, Kæledyr 
eller Camp:
1. Sørg for, batteriets opladningsniveau er over 20 %.
2. Aktiver Parkering. Indstillingerne Hold klimaanlæg tændt , 
Kæledyr , og Camp  er kun tilgængelige, når Model Y  er i 
parkeringsgear.
3. Juster om nødvendigt klimaindstillingerne.
4. På skærmbilledet med betjeningsknapper til 
klimaanlægget skal du trykke på Hold klimaanlæg tændt , 
Kæledyr  eller Camp .
BEMÆRK:  Du kan også kontrollere Kæledyr  og Camp  fra 
mobilappen ved at stryge op fra den grå bjælke på 
skærmbilledet for klimaanlægget.
Klimaanlægget forsøger at opretholde dine klimaindstillinger, 
indtil du skifter fra parkeringsgear eller deaktiverer det 
manuelt. Undgå at anvende Hold klimaanlæg tændt, 
Kæledyrstilstand eller Camp-tilstand, hvis batteriets 
opladningsniveau er lavt. Hvis batteriets opladningsniveau 
falder til under 20 %, vil Tesla-mobilappen gentagne gange 
forsøge at sende meddelelser for at minde dig om, at du skal 
kontrollere for ting, du eventuelt har efterladt i Model Y .Betjening af klimaanlægget
156 MODEL Y Instruktionsbog

For at sikre, at Model Y  giver dig den bedst mulige 
ejerskabsoplevelse i varmt vejr, skal du følge denne bedste 
praksis.
Før kørsel
Der er flere måder, hvorpå du kan forberede dit køretøj til en 
køretur, uden at du behøver sætte dig ind i et køretøj, der 
allerede er varmt:
• Klargør kabinen ved at regulere luftstrømmens retning fra 
blæserne og ved at aktivere/deaktivere sædevarmerne. I 
mobilappen skal du navigere til Klimaanlæg  for at tilpasse 
den temperatur, som du ønsker at opvarme kabinen til.
• Tryk på Planlægning , som er tilgængelig på 
skærmbillederne for både Opladning og Klimaanlæg for at 
angive et tidspunkt, hvor du gerne vil have, at dit køretøj 
er klar til en køretur (se Planlagt opladning og Planlagt 
afrejse  på side 184 ).
• Aktiver Overophedningsbeskyttelse for kabinen , som 
forhindrer, at kabinen bliver for varm i varme omgivende 
temperaturer. Du kan vælge, om du vil have 
airconditionanlægget eller blot ventilatoren til at køre, når 
temperaturen i kabinen overstiger 105° F (40° C), eller 
den valgte temperatur, såfremt denne er tilgængelig.
• I mobilappen skal du navigere til Betjeningsknapper  for at 
åbne ruderne.
BEMÆRK:  Tesla anbefaler, at klimaindstillingerne aktiveres 
mindst 30-45 minutter før afrejse (se Betjening af 
klimaanlægget  på side 154 ). Klargøringstiden afhænger af 
udendørstemperaturerne og andre faktorer. Mobilappen giver 
dig besked, når din bil har nået den ønskede 
klargøringstemperatur.
Efter kørsel
Lad Model Y  være tilsluttet opladning, når den ikke er i brug, 
særligt hvis du benytter Klargøring eller 
Overophedningsbeskyttelse for kabinen. Derved benyttes 
ladesystemet og ikke selve batteriet til at opretholde en 
behagelig temperatur (se Oplysninger om 
højspændingsbatteriet  på side 177 ). Derudover er der 
adskillige måder, hvorpå man kan afkøle en varm kabine:
• Inden du forlader køretøjet (hvis du f.eks. har et ærinde), 
kan du benytte Kæledyrstilstand til at holde kabinen kølig 
af hensyn til kæledyr eller letfordærvelige fødevarer. Se 
Hold klimaanlæg tændt, Kæledyr og Camp  på side 156  for 
yderligere oplysninger.
• Tesla anbefaler, at man deaktiverer airconditionanlægget 
ca. 30 sekunder, før man trykker på Park (P) for at 
mindske dannelsen af vandpytter under køretøjet.
• Parker i skyggen for at hjælpe med til at reducere 
strømforbruget og opretholde en køligere 
kabinetemperatur.
• Anvend en solskærm (tilgængelig via Tesla-butikken) , 
hvis det er nødvendigt at parkere udendørs i solen.• Når du holder parkeret, skal du tilslutte Model Y  og 
Planlægge  din opladning. Dit køretøj bestemmer det rette 
tidspunkt, hvor opladningen skal begynde, så den 
afsluttes i løbet af de timer, hvor elpriserne er lave. 
Kabinen og batteriet klargøres også, inden det 
afrejsetidspunkt du har angivet. For yderligere 
oplysninger henvises til Planlagt opladning og Planlagt 
afrejse  på side 184 .
Opladning
Når du benytter Turplanlægger eller navigerer til en 
Supercharger-station, så klargør dit køretøj automatisk 
batteriet med henblik på så effektiv en opladning som muligt. 
I ekstrem varme ser du muligvis ikke meddelelsen om at 
køretøjet klargør batteriet, mens du navigerer til en 
Supercharger, men køretøjet vil stadig klargøre batteriet til 
opladning.
BEMÆRK:  Tesla anbefaler, at man benytter turplanlæggeren 
til at navigere til en ladestation senest 30-45 minutter før 
ankomst for at sikre optimal batteritemperatur og 
opladningsbetingelser. Hvis det tager mindre end 30-45 
minutter at tilbagelægge afstanden til ladestationen, bør du 
overveje at klargøre batteriet, inden du kører (se Før kørsel 
på side 160 ).
Hvis det er muligt skal du lade dit køretøj være tilsluttet en 
lader, når det ikke er i brug, også i varmt vejr, særligt hvis du 
benytter Klargøring eller Overophedningsbeskyttelse for 
kabinen.
Opbevaring
Hvis du efterlader Model Y  parkeret i en længere periode, 
skal køretøjet sluttes til en oplader for at undgå normalt 
rækkeviddetab og holde batteriet ved en optimal temperatur. 
Det er sikkert at lade bilen være tilsluttet i alle tidsrum.
Når bilen ikke er i brug, går Model Y  automatisk i 
dvaletilstand for at spare energi. Reducer det antal gange, du 
kontrollerer bilens status på mobilappen, da dette aktiverer 
bilen og starter det normale energiforbrug.Bedste praksis i varmt vejr
163 Klimaanlæg

For at sikre, at Model Y  giver dig den bedst mulige oplevelse i 
koldt og barskt vejr, skal du følge denne bedste praksis.
Før kørsel
Når der samler sig sne og is på køretøjet, kan bevægelige 
dele som dørhåndtag , ruder, spejle og viskere  fryse fast. For 
at opnå maksimal rækkevidde og ydeevne hjælper det at 
opvarme kabinen og batteriet, før du kører. Der er flere måder 
at gøre det på:
• Tryk på Planlægning , som er tilgængelig på 
skærmbillederne for både opladning og klimaanlæg for at 
angive et tidspunkt, hvor du gerne vil have, at dit køretøj 
er klar til en køretur (se Planlagt opladning og Planlagt 
afrejse  på side 184 ).
• I mobilappen skal du navigere til Klimaanlæg  for at 
tilpasse den temperatur, som du ønsker at opvarme 
kabinen til. Det varmer også højspændingsbatteriet efter 
behov.
• I mobilappen skal du navigere til Klimaanlæg  > Afrim bil  for 
at optø sne, is og frost på forruden,  ladeporten,  vinduerne 
og spejlene. Det varmer også højspændingsbatteriet efter 
behov.
BEMÆRK:  Tesla anbefaler, at klimaindstillingerne 
aktiveres mindst 30-45 minutter før afrejse (se Betjening 
af klimaanlægget  på side 154 ). Klargøringstiden afhænger 
af udendørstemperaturerne og andre faktorer. 
Mobilappen giver dig besked, når din bil har nået den 
ønskede klargøringstemperatur.
Ladeport
Hvis ladeportens lås fryser fast, og et ladekabel sidder fast i 
ladeporten, skal du forsøge at løsne ladekablet manuelt. Se 
Manuel frigørelse af ladekabel  på side 182 .
I ekstremt koldt vejr eller frostvejr er der risiko for, at 
ladeportens lås fryser fast. Under disse vejrforhold kan du 
optø isen på ladeportens lås, så ladekablet kan tages ud eller 
sættes i. For at gøre dette skal du aktivere Afrim bil  via 
mobilappen. 
Du kan også forhindre at en ladeport fryser fast ved at 
benytte indstillingen Planlægning  (se Planlagt opladning og 
Planlagt afrejse  på side 184 ).
BEMÆRK:  Hvis ladeporten er frosset fast, kan den muligvis 
ikke låse ladekablet på plads, når det sættes i, men AC-
opladning lader sig stadig gøre ved en lavere hastighed, selv 
om låsen ikke er aktiveret.Opladning
Ved at bruge Turplanlægger (hvis tilgængelig)  til at navigere 
til en Tesla-ladestation, foropvarmer Model Y 
højspændingsbatteriet for at sikre, at batteriets temperatur 
er optimalt og klar til opladning, når du ankommer til 
ladestationen. Dette reducerer den tid, du skal bruge på 
opladning. Se (se Turplanlægger  på side 168 ).
BEMÆRK:  Tesla anbefaler, at man benytter turplanlæggeren 
til at navigere til en ladestation senest 30-45 minutter før 
ankomst for at sikre optimal batteritemperatur og 
opladningsbetingelser. Hvis det tager mindre end 30-45 
minutter at tilbagelægge afstanden til ladestationen, bør du 
overveje at klargøre batteriet, inden du kører (se Før kørsel 
på side 160 ).
BEMÆRK:  Varmesystemet vil muligvis udlede damp under 
visse forhold i køretøjer med varmepumpe (for at finde ud af, 
om dit køretøj er udstyret med en varmepumpe, skal du 
trykke på Betjeningsknapper  > Software  > Ekstra 
køretøjsinformation ). F.eks. kan der komme lugtløs damp fra 
fronten af dit køretøj, mens det oplader ved en Supercharger, 
hvis temperaturen er lav. Dette er normalt, og ikke noget der 
bør være årsag til bekymring.
Ruder
I mobilappen skal du navigere til Klimaanlæg  og vælge Afrim 
bil for at optø sne, is og frost på forruden, ruderne og 
spejlene.
I kolde temperaturer foretager Model Y  automatisk en mindre 
justering af vinduernes placering for at gøre det lettere at 
åbne døre i kold temperatur.
BEMÆRK:  Tilslut altid til en ekstern lavspænding -
strømforsyning, før du åbner en dør, når køretøjet er afladet, 
for at undgå at knuse en rude.
Brug mobilappen til at planlægge en serviceaftale, hvor Tesla 
kan påføre en vandafvisende belægning på sideruderne og 
bagruderne (ikke forruden) for et symbolsk beløb.
Døre
I barske vinterforhold kan isdannelse gøre det sværere at 
åbne dørhåndtagene. Du kan benytte mobilappen til åbne 
døren i førersiden i denne situation.
1. I mobilappen skal du trykke på en af de fire 
genvejsknapper og holde den nede og følge 
instruktionerne for at tilpasse genvejene med Lås op for 
dør.
2. Når du står ved siden af din bil, skal du trykke på Lås op 
for dør  for at åbne førerdøren.Bedste praksis i koldt vejr
160 MODEL Y Instruktionsbog

BEMÆRK:  For at gøre det nemt for dig giver Tesla dig 
mulighed for at vælge mellem en række forskellige 
sprogindstillinger til stemmekommandoerne. For at vælge et 
andet sprog skal du trykke på Betjeningsknapper  > Display  > 
Sprog for stemmegenkendelse .
Benyt stemmekommandoer til nem justering af indstillinger 
og præferencer uden at skulle bruge touchskærmen. 
Stemmekommandoer er designet til at forstå naturlige 
anmodninger. Nedenstående er ikke en udtømmende liste 
over handlinger, du kan udføre ved hjælp af 
stemmekommandoer:
• Justere dine præferencer for klimaanlægget
• Justere vinduesviskernes hastighed og frekvens
• Styre forskellige dele af dit køretøj
• Navigere til en lokation
• Ringe til en kontaktperson
• Interagere med apper og indstillinger
For at indlede en stemmekommando skal du trykke på højre 
rulleknap på rat og slippe den igen . Når der lyder en 
klokkelyd, skal du komme med din anmodning.
Eksempler på stemmekommandoer
Her er en liste med eksempler på stemmekommandoer. Dette 
er ikke en udtømmende liste. Tesla arbejder løbende på at 
forbedre stemmekommandoer.
BEMÆRK:  Dit køretøj skal stå i parkeringsgear (P) for at 
aktivere visse stemmekommandoer (f.eks. Sentry Mode, 
Kæledyrstilstand osv.).
Betjening af klimaanlægget
Juster dine præferencer for klimaanlægget:
• "Gør det koldere"• "Gør det varmere"
• "Aktiver/deaktiver varmen i førersædet"
• "Afkøl passageren"
• "Ret luftstrømmen mod mit ansigt"
• "Synkroniser klima"
• "Forøg/reducer blæserhastigheden"
• "Aktivér/deaktiver afrimer"
• "Indstil temperatur/ventilator ..."
• "Aktiver recirkulering"
Forrudevisker e
Opdater vinduesviskernes hastighed og frekvens baseret på 
ændringer i vej- og vejrforhold:
• "Forøg vinduesviskernes hastighed"
• "Forøg/reducer vinduesviskernes hastighed med ..."
• "Aktiver/deaktiver vinduesviskerne"
Køretøjets betjeningselementer
Ændre forskellige betjeningselementer i dit køretøj:
• "Aktiver/deaktiver Sentry-tilstand"
• "Sørg for, at min bil er sikker"
• "Lås/lås op for dørene"
• Aktiver Kæledyrstilstand"
• "Fold sidespejle ind/ud"
• "Åbn/luk ladeport"
• "Start/stop opladning"
• "Åbn indstillinger for service"
• "Åbn handskerummet"
Navigation
Søg efter eller naviger til en lokation:
• "Hvor er [lokation]?"
• "Kør til [lokation]"
• "Naviger til [lokation]"
• "Vis Superchargere i nærheden"
• "Jeg føler mig sulten/heldig"  (se Kort og navigation  på 
side 164 ).
• "Stop navigation"
• "Deaktiver stemmevejledning"Stemmekommandoer
18 MODEL Y Instruktionsbog

-----------------------------------



In [25]:

stream = openai_client.chat.completions.create(
    messages=[{"role": "user", "content": query}],
    model="gpt4",
    stream=True)

output = ""
for chunk in stream:
    if chunk.choices:  # Check if the list is not empty
        output += chunk.choices[0].delta.content or ""
    display(Markdown(f"{output}"), clear=True)

Ændring af temperaturen i en Tesla er enkel og kan gøres via bilens touchskærm.

1. På touchskærmen skal du trykke på fan-ikonet nederst på skærmen.
2. Dette vil åbne klimakontrolpanelet. Her kan du justere temperaturindstillingerne.
3. For at øge temperaturen skal du trykke på "+" symbolet ved siden af det nuværende temperaturtal.
4. For at sænke temperaturen skal du trykke på "-" symbolet.
5. Hvis bilen har Dual Zone Climate Control, kan du justere fører- og passagersiden separat ved at trykke på "SYNC" knappen for at deaktivere synkroniseringen, og derefter justere hver side individuelt.
6. Når du er færdig, skal du trykke på "X" i hjørnet for at lukke klimakontrolpanelet.

Bemærk, at indstillingerne kan variere lidt afhængigt af din Tesla-model og softwareversion. Du kan også bruge Tesla-appen på din smartphone til at kontrollere klimaet på afstand, hvilket kan være praktisk, hvis du vil opvarme eller køle bilen før du kommer i den.

## Normal RAG example with llamaindex

In [26]:
import chromadb
import os

from dotenv import load_dotenv
from chromadb import Settings
from llama_index.llms.openai import OpenAI
from llama_index.core import SimpleDirectoryReader, VectorStoreIndex
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core.ingestion import IngestionPipeline
from llama_index.embeddings.openai import OpenAIEmbedding
# %pip install llama-index

from llama_index.vector_stores.chroma import ChromaVectorStore

from llama_index.llms.azure_openai import AzureOpenAI
from llama_index.embeddings.azure_openai import AzureOpenAIEmbedding

load_dotenv()

# ChromaDB Vector Store
chroma_client = chromadb.PersistentClient(
    path="./chromadb", settings=Settings(allow_reset=True))
chroma_client.reset()
collection = chroma_client.get_or_create_collection(
    name="tesla", metadata={"hnsw:space": "cosine"})
vector_store = ChromaVectorStore(chroma_collection=collection)


llm = AzureOpenAI(
    model="gpt-4",
    deployment_name="gpt4",
    api_key=os.getenv("OPENAI_API_KEY"),  
    api_version=os.getenv("OPENAI_API_VERSION"), # https://learn.microsoft.com/en-us/azure/ai-services/openai/reference?WT.mc_id=AZ-MVP-5004796
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT")
)

# You need to deploy your own embedding model as well as your own chat completion model
embedding = AzureOpenAIEmbedding(
    model="text-embedding-ada-002",
    deployment_name="text-embedding-ada-002",
    api_key=os.getenv("OPENAI_API_KEY"),  
    api_version=os.getenv("OPENAI_API_VERSION"), # https://learn.microsoft.com/en-us/azure/ai-services/openai/reference?WT.mc_id=AZ-MVP-5004796
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT")
)

# Define the ingestion pipeline to add documents to vector store
pipeline = IngestionPipeline(
    transformations=[
        SentenceSplitter(chunk_size=512, chunk_overlap=20),
        embedding,
    ],
    vector_store=vector_store,
)

# Create index with the vector store and using the embedding model
index = VectorStoreIndex.from_vector_store(
    vector_store=vector_store, embed_model=embedding)

In [27]:
# Fetch documents
documents = SimpleDirectoryReader('./documents').load_data()

# Run pipeline
pipeline.run(documents=documents)

print("Done")

Done


In [38]:
from llama_index.core import PromptTemplate
from llama_index.core.query_engine import CustomQueryEngine
from llama_index.core.retrievers import BaseRetriever
from llama_index.core import get_response_synthesizer
from llama_index.core.response_synthesizers import BaseSynthesizer

    
qa_prompt = PromptTemplate(
    """You are a helpful assistant that answers questions about my Tesla Model Y using provided context from the owner's manual. 
    You must provide your answer in the Danish language. 
    The answer should be in a bullet format.
    You should provide the number of the page containing the information given in the answer. The page number should be in the format "Side x after an empty line".
    Omit the metadata from {context_str} and only provide the content of the context.
    ---------------------
    {context_str}
    ---------------------
    Given the context information and not prior knowledge, answer the query.
    Query: {query_str}
    Answer: 
    """,
)


class RAGQueryEngine(CustomQueryEngine):
    """RAG String Query Engine."""

    retriever: BaseRetriever
    response_synthesizer: BaseSynthesizer
    llm: OpenAI
    qa_prompt: PromptTemplate

    def custom_query(self, query_str: str):
        nodes = self.retriever.retrieve(query_str)
        context_str = "\n\n".join([n.node.get_content(metadata_mode="all") for n in nodes])
        #context = qa_prompt.format(
        #    context_str=context_str, query_str=query_str)
        response = self.llm.complete(
            qa_prompt.format(context_str=context_str, query_str=query_str)
        )
                    
        return str(response) + "\n\n-------------------------\n\nKontekst:\n\n" + context_str


synthesizer = get_response_synthesizer(response_mode="compact")
query_engine = RAGQueryEngine(
    retriever=index.as_retriever(),
    response_synthesizer=synthesizer,
    llm=llm,
    qa_prompt=qa_prompt,
)

In [40]:
query = "hvordan lader jeg min Tesla Model Y?"

from IPython.display import Markdown, display

response = query_engine.query(query)
display(Markdown(f"{response}"))

- Sørg for, at Model Y er i parkeringsgear.
- Få adgang til ladeindstillinger ved at trykke på Betjeningsknapper > Opladning, når Model Y er i parkeringsgear. Du kan også trykke på batteriikonet på touchskærmen for at få adgang til ladeindstillingerne.
- Køreafstand: Viser den samlede estimerede køreafstand.
- Indstil grænse: Juster opladningsskydeknappen til det opladningsniveau, du ønsker. Den indstilling, du vælger, anvendes til aktuelle og planlagte opladninger.

Side 181

-------------------------

Kontekst:

page_label: 219
file_name: Owners_Manual_model_y.pdf
file_path: /Users/peerchristensen/Desktop/Projects/advanced-rag-examples/tesla/documents/Owners_Manual_model_y.pdf
file_type: application/pdf
file_size: 9335241
creation_date: 2024-06-14
last_modified_date: 2024-06-14

1. Sørg for, at Model Y  er i parkeringsgear.
2. Kontrollér dækkene, eller sørg for, at Model Y  er sikker.
3. Træd bremsepedalen ned, hold den nede og tryk derefter 
på følgende på touchskærmen Betjeningsknapper  > 
Service  > Bugsering . Touchskærmen viser en meddelelse, 
der minder dig om, hvordan Model Y  skal transporteres 
korrekt.
4. Tryk på knappen Transporttilstand . Du kan se, at 
Transporttilstand er aktiveret, fordi denne knap er blå. 
Model Y  kan nu rulle frit og kan rulle langsomt (ikke 
hurtigere end gåtempo) eller trækkes.
Hvis du vil annullere transporttilstand, skal du sætte Model Y 
i parkeringsgear.
BEMÆRK:  Hvis dit køretøj er udstyret med et blysyrebatteri 
med lavspænding (se Brug af startkabler  på side 227 ): 
Transporttilstand kan blive annulleret, hvis Model Y  mister 
lavspænding -strømforsyningen efter transporttilstand er 
blevet aktiveret.
PAS PÅ:  Hvis det elektriske system ikke fungerer, og du 
derfor ikke kan benytte touchskærmen til at aktivere 
transporttilstand, skal du benytte selvlæssende 
transportvogne eller dækruller. Først skal du dog 
kontrollere producentens specifikationer og 
produktets anbefalede lastkapacitet.

page_label: 181
file_name: Owners_Manual_model_y.pdf
file_path: /Users/peerchristensen/Desktop/Projects/advanced-rag-examples/tesla/documents/Owners_Manual_model_y.pdf
file_type: application/pdf
file_size: 9335241
creation_date: 2024-06-14
last_modified_date: 2024-06-14

Ladeindstillinger
Du får adgang til ladeindstillinger ved at trykke på 
Betjeningsknapper  > Opladning , når Model Y  er i 
parkeringsgear. Du kan også trykke på batteriikonet på 
touchskærmen for at få adgang til ladeindstillingerne.
1. Køreafstand:  Viser den samlede estimerede køreafstand.
2. Indstil grænse:  Juster opladningsskydeknappen til det 
opladningsniveau, du ønsker. Den indstilling, du vælger, 
anvendes til aktuelle og planlagte opladninger.Opladningsanvisninger
181 Opladning og energiforbrug